# Fragment Decomposition

Abhinav Madahar <abhinav.madahar@rutgers.edu>, James Abello Monedero <abelloj@cs.rutgers.edu>

<br />

We want to find the fragment decomposition of a large graph.

In [46]:
import networkx as nx
import matplotlib.pyplot as plt
from queue import Queue
from unionfind import UnionFind

First, we need to be able to run pBFS on a path in a graph and get the connected components of the waves.
Let's make an implementation of pBFS which yields tuples of connected components at every iteration.

In [47]:
from collections import namedtuple


Node = namedtuple('Node', ['uid', 'children'])

def parallel_bfs(G: nx.Graph, path: list):
    queue = Queue()
    for source in path:
        queue.put(source)
    
    roots = (Node(source, []) for source in path)
    tree = {source: root for source, root in zip(path, roots)}
    
    while not queue.empty():
        v = queue.get()
        for w in G.adj[v]:
            if w not in tree:
                tree[w] = Node(w, [])
                tree[v].children.append(w)
                queue.put(w)
    
    def get_underlying_subtree(head: Node) -> Node:
        return Node(head.uid, [get_underlying_subtree(tree[child]) for child in head.children])
    
    pbfs = [get_underlying_subtree(tree[source]) for source in path]
    
    waves = {}

    def add_to_tree(head, parent: int = None, depth: int = 0):
        if depth not in waves:
            waves[depth] = []
        waves[depth].append((head.uid, parent))
        for child in head.children:
            add_to_tree(child, head.uid, depth + 1)

    for pbfs_section in pbfs:
        add_to_tree(pbfs_section)
        
    connected_components = []
    for wave in waves.values():
        uf = UnionFind()
        nodes = set(node for node, _ in wave)
        for src in nodes:
            for dest in G.adj[src]:
                if dest in nodes:
                    uf.union(src, dest)
        connected_components.append(uf.components())

    return waves, connected_components

Let's get the waves.

In [58]:
%%time
G = nx.gnm_random_graph(50000, 1300000)
G = G.subgraph(next(nx.connected_components(G))).copy()
G.add_edge(0, 1)
G.add_edge(1, 2)
G.add_edge(2, 3)
G.add_edge(3, 4)
pbfs, waves = parallel_bfs(G, [0, 1, 2, 3, 4])

CPU times: user 18.3 s, sys: 556 ms, total: 18.8 s
Wall time: 18.8 s


In [61]:
(1300000 / 50000) /  

26.0

In [57]:
from random import random

tree = nx.DiGraph()
! rm -f pbfs.dot

random_color = lambda: "%s %s %s"%(random(), random(), random())
colours = {}
for wave in waves:
    for comp in wave:
        colour = random_color()
        for node in comp:
            colours[node] = colour

with open('pbfs.dot', 'a') as pbfs_file:
    pbfs_file.write('digraph G {')
    for layer in pbfs.values():
        for dest, src in layer:
            if src:
                pbfs_file.write(f'{src} [fontcolor="{colours[src] if src in colours else "black"}"]; {dest} [fontcolor="{colours[dest] if dest in colours else "black"}"]; {src} -> {dest};')
    pbfs_file.write('}')

!dot -Tpng pbfs.dot -o pbfs.png

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.0730607 to fit
